## 1. looking at 'simplified data' input

In [1]:
import pandas as pd

import numpy as np
import json

In [2]:
file_path = "raw_data/full_simplified_The Eiffel Tower.ndjson"

In [3]:
!pwd

/home/honor/code/rs-uk/pictionary-ai/notebooks


In [4]:
#loading jason file
with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        json_line = json.loads(line)
        print(type(json_line))
        print(json_line)
        if i > 5:
            break

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/full_simplified_The Eiffel Tower.ndjson'

# instals

In [16]:
pip install tensorflow

  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from keras import Model, Sequential, layers, regularizers, optimizers
from sklearn.preprocessing import TargetEncoder
from colorama import Fore, Style

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [4]:
from sklearn.preprocessing import LabelBinarizer

In [5]:
import pandas as pd
import numpy as np

In [6]:
from typing import Tuple

# padding

In [9]:
def padding(tensor, max_length, padding='post'):
    '''function is going to take in the tensor and padd the data to max_length, return a tensor that is padded, defult is post padding'''
    padded_tensor =utils.pad_sequence(tensor, maxlen=max_length, padding=padding)
    return padded_tensor

# load data from csv

In [19]:
Y = pd.read_csv('/home/honor/code/rs-uk/pictionary-ai/raw_data/data_y.csv')
X = pd.read_csv("/home/honor/code/rs-uk/pictionary-ai/raw_data/data_X.csv")

KeyboardInterrupt: 

In [20]:
import numpy as np

In [26]:
#this function worked for X dunno what it does
XX = np.genfromtxt("../raw_data/data_X.csv", delimiter=",")
XX = XX.reshape((14352,150,3))

In [63]:
#this function worked for y as the one above wasnt working, has to have delimiter and dtype
yy =np.loadtxt("../raw_data/data_y.csv", dtype=str, delimiter=",")
yy.shape

(14352,)

In [65]:
yy[-5:]

array(["'The Great Wall of China'", "'The Great Wall of China'",
       "'The Great Wall of China'", "'The Great Wall of China'",
       "'The Great Wall of China'"], dtype='<U25')

In [55]:
#works but gives wierd format
Y = pd.read_csv('/home/honor/code/rs-uk/pictionary-ai/raw_data/data_y.csv').T

In [61]:
Y.columns = ['target']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [36]:
#this was when using genfromtxt
yy

array([nan, nan, nan, ..., nan, nan, nan])

In [25]:
XX.shape

(6458400,)

# target encoder

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [70]:
target_encoder = OneHotEncoder(sparse_output=False)
#terget encoding, than transforming y which is the classes
y_encoded =target_encoder.fit_transform(yy.reshape(-1,1))

In [71]:
y_encoded.shape

(14352, 2)

# split data

In [72]:
padded_tensor = XX

tensor_length = len(padded_tensor)
train_length = int(0.7 * tensor_length)
test_length = tensor_length- train_length

#taking in the padded X data and splititng it 70 30
X_train = padded_tensor[:train_length,]
X_test = padded_tensor[train_length:,]

#taking in y encoded and spliting it 70 30
y_train = y_encoded[:train_length]
y_test = y_encoded[train_length:]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

# make models

In [12]:
# this is the length we are padding too
max_length = 150

#no of classes we are using
num_classes = 10
def model_bidirectional():
    """
    Initialize the Neural Network with random weights, using bidirectional LTSM
    masking layer
    it has 2 Bidirectional LSTM layers
    3 dense layers
    and dropout layers
    """

    model = Sequential()

    # Add Masking layer to handle variable-length sequences
    #put in 99 as 0 may effect the data
    model.add(layers.Masking(mask_value=99, input_shape=(max_length, 3)))

    #do we want to customize backwards layer?

    model.add(layers.Bidirectional(layers.LSTM(196, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))

    # Add Dense layers
    model.add(layers.Dense(128, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(64, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(32, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))

    # Add final Softmax layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    # Replace 'num_classes' with the actual number of classes in your problem

    print("✅ Model initialized")

    return model

def model_LTSM():
    """
    Initialize the Neural Network with random weights
    model that just has LSTM same structure otherwise
    """

    model = Sequential()

    # Add Masking layer to handle variable-length sequences
    #put in 99 as 0 may effect the data
    model.add(layers.Masking(mask_value=99, input_shape=(max_length, 3)))

    # Add LSTM layers
    model.add(layers.LSTM(64, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

    model.add(layers.LSTM(32, activation='tanh', dropout=0.2, recurrent_dropout=0.2))


    # Add Dense layers
    model.add(layers.Dense(128, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(64, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(32, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))

    # Add final Softmax layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    # Replace 'num_classes' with the actual number of classes in your problem

    print("✅ Model initialized")

    return model

def model_LTSM_conv():
      '''model has conv1d layer and max pooling adn than LTSM, got this model from
      https://medium.com/@www.seymour/training-a-recurrent-neural-network-to-recognise-sketches-in-a-real-time-game-of-pictionary-16c91e185ce6'''
      model = Sequential()

      # Input layer
      model.add(layers.Masking(mask_value=99, input_shape=(max_length, 3)))


      # 1D Convolutional Layers- should i have more or less dropout?
      model.add(layers.Conv1D(32, 3, activation='relu'))
      model.add(layers.Conv1D(64, 3, activation='relu'))
      model.add(layers.MaxPooling1D(2))
      model.add(layers.Dropout(rate=0.2))

      model.add(layers.Conv1D(128, 3, activation='relu'))
      model.add(layers.MaxPooling1D(2))
      model.add(layers.Dropout(rate=0.2))

      # Recurrent layers (e.g., LSTM)
      model.add(layers.LSTM(128, return_sequences=True,dropout=0.2, recurrent_dropout=0.2))

      model.add(layers.LSTM(128,dropout=0.2, recurrent_dropout=0.2))


      # Dense layers
      model.add(layers.Dense(128, activation='relu'))
      model.add(layers.Dropout(rate=0.2))
      model.add(layers.Dense(128, activation='relu'))
      model.add(layers.Dropout(rate=0.2))

      # Output layer
      model.add(layers.Dense(num_classes, activation='softmax'))

      return model

def compile_model(model: Model, learning_rate=0.0005):
    """
    Compile the Neural Network
    with loss categorical_crossentropy, optimiser adam, metrics, accuracy
    """
    #what loss do we want?
    #i think should be using categorical
    #which metrics?
    #do i want to create my own and what are the advantages of this

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    # look at custum loss function
    print("✅ Model compiled")

    return model

def train_model(
        model: Model,
        X: np.ndarray,
        y: np.ndarray,
        batch_size=256,
        patience=3,
        validation_data=None, # overrides validation_split
        validation_split=0.3
    ):
    """
    Fit the model and return a tuple (fitted_model, history)
    added in checkpoint as well
    """
    print(Fore.BLUE + "\nTraining model..." + Style.RESET_ALL)

    es = callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=patience,
        restore_best_weights=True,
        verbose=1
    )

    checkpoint_filepath = '/home/honor/code/rs-uk/pictionary-ai/raw_data/models'
    #this will save the checkpoints in the checkpoint_filepath
    model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

    #in fit is where we put in the padding, cant remember how
    history = model.fit(
        X,
        y,
        validation_data=validation_data,
        validation_split=validation_split,
        epochs=50,
        batch_size=batch_size,
        callbacks=[es, model_checkpoint_callback],
        verbose=1
    )

    print(f"✅ Model trained on {len(X)} rows with min val accuracy: {round(np.min(history.history['accuracy']), 2)}")

    return model, history


def evaluate_model(
        model: Model,
        X: np.ndarray,
        y: np.ndarray,
        batch_size=64
    ) -> Tuple[Model, dict]:
    """
    Evaluate trained model performance on the dataset
    """

    print(Fore.BLUE + f"\nEvaluating model on {len(X)} rows..." + Style.RESET_ALL)

    if model is None:
        print(f"\n❌ No model to evaluate")
        return None

    metrics = model.evaluate(
        x=X,
        y=y,
        batch_size=batch_size,
        verbose=0,
        # callbacks=None,
        return_dict=True
    )

    loss = metrics["loss"]
    mae = metrics["accuracy"]

    print(f"✅ Model evaluated, accuracy: {round(mae, 2)}")

    return metrics

NameError: name 'Model' is not defined

# use model

In [83]:
#initialize model
model = model_bidirectional()
#compile model
model= compile_model(model)
#train model
model, history = train_model(model, X_train, y_train, validation_data=[X_val,y_val])
#evaluate model
metrics=evaluate_model(model, X_test, y_test)

✅ Model initialized
✅ Model compiled

Training model...
Epoch 1/50
32/32 [==============================] - 85s 2s/step - loss: 0.1180 - accuracy: 0.9822 - val_loss: 1.3789e-04 - val_accuracy: 1.0000
Epoch 2/50
32/32 [==============================] - 77s 2s/step - loss: 2.9557e-05 - accuracy: 1.0000 - val_loss: 9.7442e-05 - val_accuracy: 1.0000
Epoch 3/50
32/32 [==============================] - 81s 3s/step - loss: 3.7823e-05 - accuracy: 1.0000 - val_loss: 9.4778e-05 - val_accuracy: 1.0000
Epoch 4/50
32/32 [==============================] - 151s 5s/step - loss: 1.6849e-05 - accuracy: 1.0000 - val_loss: 9.3530e-05 - val_accuracy: 1.0000
Epoch 4: early stopping
✅ Model trained on 8036 rows with min val accuracy: 0.98

Evaluating model on 4306 rows...
✅ Model evaluated, accuracy: 0.8


In [84]:
#initialize model
model2 = model_LTSM()
#compile model
model2= compile_model(model2)
#train model
model2, history = train_model(model2, X_train, y_train, validation_data=[X_val,y_val])
#evaluate model
metrics=evaluate_model(model2, X_test, y_test)

ERROR! Session/line number was not unique in database. History logging moved to new session 195
✅ Model initialized
✅ Model compiled

Training model...
Epoch 1/50
32/32 [==============================] - 24s 479ms/step - loss: 0.2597 - accuracy: 0.9780 - val_loss: 8.0146e-04 - val_accuracy: 1.0000
Epoch 2/50
32/32 [==============================] - 14s 442ms/step - loss: 2.9099e-04 - accuracy: 1.0000 - val_loss: 2.4475e-04 - val_accuracy: 1.0000
Epoch 3/50
32/32 [==============================] - 15s 458ms/step - loss: 1.5092e-04 - accuracy: 1.0000 - val_loss: 2.2709e-04 - val_accuracy: 1.0000
Epoch 4/50
32/32 [==============================] - 14s 430ms/step - loss: 1.4613e-04 - accuracy: 1.0000 - val_loss: 2.1984e-04 - val_accuracy: 1.0000
Epoch 4: early stopping
✅ Model trained on 8036 rows with min val accuracy: 0.98

Evaluating model on 4306 rows...
✅ Model evaluated, accuracy: 0.8


In [89]:
#initialize model
model3 = model_LTSM_conv()
#compile model
model3= compile_model(model3)
#train model
model3, history = train_model(model3, X_train, y_train, validation_data=[X_val,y_val])
#evaluate model
metrics=evaluate_model(model3, X_test, y_test)

✅ Model compiled

Training model...
Epoch 1/50
32/32 [==============================] - 17s 272ms/step - loss: 0.2188 - accuracy: 0.9716 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
32/32 [==============================] - 7s 223ms/step - loss: 1.4322e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
32/32 [==============================] - 9s 271ms/step - loss: 6.8089e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
32/32 [==============================] - 10s 302ms/step - loss: 5.0466e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4: early stopping
✅ Model trained on 8036 rows with min val accuracy: 0.97

Evaluating model on 4306 rows...
✅ Model evaluated, accuracy: 0.8


observations
- all have similiar accuracy, this si expected as only 2 classes
- go in order for trianing times

# loading data

In [27]:
!pip install ndjson

In [17]:
file_url = '../raw_data/trombone.ndjson'

In [39]:
import ndjson

with open(file_url,'r') as f:
    for line in f:

        ndjson.loads(line)

        line_fixed = eval(line.replace('true','True'))

        print(line_fixed['word'])

        break

trombone


In [10]:
def model_bidirectional2():
    """
    Initialize the Neural Network with random weights, using bidirectional LTSM
    masking layer
    it has 2 Bidirectional LSTM layers
    3 dense layers
    and dropout layers
    """

    model = Sequential()

    # Add Masking layer to handle variable-length sequences
    #put in 99 as 0 may effect the data
    model.add(layers.Masking(mask_value=99, input_shape=( max_length, 3)))

    #do we want to customize backwards layer?

    model.add(layers.Bidirectional(layers.LSTM(196, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))

    # Add Dense layers
    model.add(layers.Dense(128, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(64, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(32, activation='linear'))
    #dropoutlayer
    model.add(layers.Dropout(rate=0.2))

    # Add final Softmax layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    # Replace 'num_classes' with the actual number of classes in your problem

    print("✅ Model initialized")

    return model

In [7]:
model1 =model_bidirectional2()

model1 = compile_model(model1)


2024-03-08 15:11:55.448804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-08 15:11:55.448856: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-08 15:11:55.448889: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovo): /proc/driver/nvidia/version does not exist
2024-03-08 15:11:55.449245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


✅ Model initialized
✅ Model compiled


In [8]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 150, 3)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 150, 392)         313600    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              233984    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8

In [15]:
#called the folder and than what the model is called
model1.load_weights('../raw_data/models/models')

In [24]:
test = list([[[0,22,37,64,255],[218,220,227,228,211]],[[76,95,135,141,150,159,166,180,186,201],[220,138,31,0,63,79,117,150,191,224]],[[94,104,111,119,127,141,143,142,180,191],[212,167,149,80,59,41,30,134,202,232]],[[109,127,137,147,150,162,172,185],[122,120,104,97,99,124,128,128]],[[75,130,158],[162,159,150]]])

In [25]:

def process_drawing_data(json_drawing: json) -> np.array:
    '''
    Extracts the drawing data (strokes list) from a drawing JSON file.
    Transforms the strokes from coordinates to deltas.
    Returns an np.array of deltas (d_x, d_y, end_of_stroke)
    '''
    # --- Data extraction ---
    list_strokes = json_drawing

    x = []
    y = []
    stroke_delimiter = []
    list_points = [x, y, stroke_delimiter]

    for stroke in list_strokes:
        # Creating the third list to pass to the model with 0 all along a stroke and a 1 at the end of the stroke
        stroke_delimiter = [0.] * len(stroke[0])
        stroke_delimiter[-1] = 1
        # Concatenating x, y, and the delimiter to the new list of points
        list_points[0] += stroke[0]
        list_points[1] += stroke[1]
        list_points[2] += stroke_delimiter

    np_points = np.asarray(list_points)
    np_points = np_points.T

    # --- Processing ---
    # 1. Size normalization
    lower = np.min(np_points[:, 0:2], axis=0) # returns (x_min, y_min)
    upper = np.max(np_points[:, 0:2], axis=0) # returns (x_max, y_max)
    scale = upper - lower # returns (width, heigth)
    scale[scale == 0] = 1 # to escape a zero division for a vertical or horizontal stroke
    np_points[:, 0:2] = (np_points[:, 0:2] - lower) / scale

    # 2. Compute deltas
    np_points[1:, 0:2] -= np_points[0:-1, 0:2]
    np_points = np_points[1:, :]

    return np.round(np_points,decimals=4)

In [26]:
eifell_tower = process_drawing_data(test)

len(eifell_tower)

35

In [27]:
def padded_tensor(tensor) :
    if len(tensor) >= 150 :
        return tensor[0:150]
    else :
        pad_length = 150 - len(tensor)
        padding = [[99,99,99]]
        return np.concatenate((tensor,np.array(padding*pad_length)),axis=0)

In [33]:
X_eifell = np.expand_dims(padded_tensor(eifell_tower),0)

X_eifell.shape

(1, 150, 3)

In [36]:
res = model1.predict(X_eifell)[0]

1/1 [==============================] - 0s 120ms/step


In [38]:
np.argmax(res)

0

In [37]:
import requests

predict_url = "http://localhost:8080/api"

In [38]:
test = list([[[0,22,37,64,255],[218,220,227,228,211]],[[76,95,135,141,150,159,166,180,186,201],[220,138,31,0,63,79,117,150,191,224]],[[94,104,111,119,127,141,143,142,180,191],[212,167,149,80,59,41,30,134,202,232]],[[109,127,137,147,150,162,172,185],[122,120,104,97,99,124,128,128]],[[75,130,158],[162,159,150]]])

In [51]:
post_dict = {"drawing":[[[124,123,119,119,119,120,122,127,132,140,152,166,184,207,224,238,246,249,250,249,240,228,220,207,170,144,137,130,109,90],[302,301,294,289,283,278,270,264,258,253,248,246,245,245,245,249,253,257,260,265,280,296,300,301,303,306,307,307,305,305]],[[248,249,270,297,313,323,329,336,339,342,342,341,334,328,321,307,293,285,278,271,262,256],[252,251,245,243,244,247,251,257,261,268,275,279,287,292,294,292,287,282,277,270,262,258]],[[340,340,345,356,368,377,383,387,389,389,388,383,375,367,347,329],[246,244,236,228,221,224,229,236,247,253,255,257,258,259,259,257]],[[354,354,354,354],[223,221,207,195]],[[374,374,376],[218,213,207]],[[145],[302]],[[182,182,182,182],[284,288,293,312]],[[146,146],[295,299]],[[227,227,228,230,239],[295,305,312,315,325]],[[323,323,323],[281,286,291]],[[326,326,326,328,336],[291,295,299,306,322]],[[299,299,299,299,296,292,290,289,288,286],[277,281,286,290,296,305,310,313,315,319]]]}








res = requests.post(url=predict_url, json=post_dict, headers={'Content-Type':'application/json'})

In [52]:
res = res.content

In [55]:
res


b'{"result":"[0.00135539 0.00861962 0.00254553 0.00099782 0.00162312 0.00282414\\n 0.00431927 0.01727205 0.0419828  0.91846025]","prediction":"9"}'

In [53]:
#decode turns to string, eval turns into dictionary
eval(res.decode())['prediction']

'9'

In [54]:
dict_10_classes = {0: 'The Eiffel Tower',
 1: 'The Great Wall of China',
 2: 'The Mona Lisa',
 3: 'aircraft carrier',
 4: 'airplane',
 5: 'alarm clock',
 6: 'ambulance',
 7: 'angel',
 8: 'animal migration',
 9: 'ant'}

print(dict_10_classes[int(eval(res.decode())['prediction'])])

ant


In [28]:
res = requests.post(url="http://localhost:8080/api")

In [13]:
res.content

b'"Welcome to the pictionary ai api"'

In [46]:
dict_10_classes = {0: 'The Eiffel Tower',
 1: 'The Great Wall of China',
 2: 'The Mona Lisa',
 3: 'aircraft carrier',
 4: 'airplane',
 5: 'alarm clock',
 6: 'ambulance',
 7: 'angel',
 8: 'animal migration',
 9: 'ant'}

In [56]:
dictionary = {"aircraft carrier": 0, "arm": 1, "asparagus": 2, "backpack": 3,
              "banana": 4, "basketball": 5, "bottlecap": 6, "bread": 7, "broom": 8,
              "bulldozer": 9, "butterfly": 10, "camel": 11, "canoe": 12, "chair": 13,
              "compass": 14, "cookie": 15, "drums": 16, "eyeglasses": 17, "face": 18,
              "fan": 19, "fence": 20, "fish": 21, "flying saucer": 22, "grapes": 23,
              "hand": 24, "hat": 25, "horse": 26, "light bulb": 27, "lighthouse": 28,
              "line": 29, "marker": 30, "mountain": 31, "mouse": 32, "parachute": 33,
              "passport": 34, "pliers": 35, "potato": 36, "sea turtle": 37, "snowflake": 38,
              "spider": 39, "square": 40, "steak": 41, "swing set": 42, "sword": 43,
              "telephone": 44, "television": 45, "tooth": 46, "traffic light": 47, "trumpet": 48, "violin": 49}

In [57]:
reversed_dict = {v: k for k, v in dictionary.items()}

In [58]:
reversed_dict

{0: 'aircraft carrier',
 1: 'arm',
 2: 'asparagus',
 3: 'backpack',
 4: 'banana',
 5: 'basketball',
 6: 'bottlecap',
 7: 'bread',
 8: 'broom',
 9: 'bulldozer',
 10: 'butterfly',
 11: 'camel',
 12: 'canoe',
 13: 'chair',
 14: 'compass',
 15: 'cookie',
 16: 'drums',
 17: 'eyeglasses',
 18: 'face',
 19: 'fan',
 20: 'fence',
 21: 'fish',
 22: 'flying saucer',
 23: 'grapes',
 24: 'hand',
 25: 'hat',
 26: 'horse',
 27: 'light bulb',
 28: 'lighthouse',
 29: 'line',
 30: 'marker',
 31: 'mountain',
 32: 'mouse',
 33: 'parachute',
 34: 'passport',
 35: 'pliers',
 36: 'potato',
 37: 'sea turtle',
 38: 'snowflake',
 39: 'spider',
 40: 'square',
 41: 'steak',
 42: 'swing set',
 43: 'sword',
 44: 'telephone',
 45: 'television',
 46: 'tooth',
 47: 'traffic light',
 48: 'trumpet',
 49: 'violin'}

In [60]:
import random
random.choice(list(reversed_dict.values()))

'face'

In [15]:

# initiate model
model = model_bidirectional()
model = compile_model(model)
#load wieghts
model.load_weights('../raw_data/models/models_1003_50classes')

NameError: name 'Sequential' is not defined